# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/WWF-Bird-Recognition-Project/

/content/drive/MyDrive/WWF-Bird-Recognition-Project


In [ ]:
!git config --global user.email 'ah05084@st.habib.edu.pk'
!git config --global user.name 'hurryingauto3'

In [9]:
!git pull 

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/hurryingauto3/WWF-Bird-Recognition-Project
   1c58e3b..e8e9070  main       -> origin/main
Updating 1c58e3b..e8e9070
Fast-forward
 .gitignore | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
!git add . && git commit -m "Notebook Updated" && git push 

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, applications, optimizers
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
!unzip '/content/drive/MyDrive/WWF-Bird-Recognition-Project/WWF-Khidmat/dataset100.zip'

In [ ]:
# def load_dataset(size):
def load_dataset():
  # if size == 100:

  #   cm = [x for x in os.walk('CM100')][0][2]
  #   hs = [x for x in os.walk('HS100')][0][2]
  #   hc = [x for x in os.walk('HC100')][0][2]

  # elif size == 200:
  #   cm = [x for x in os.walk('CM200')][0][2]
  #   hs = [x for x in os.walk('HS200')][0][2]
  #   hc = [x for x in os.walk('HC200')][0][2]
  
  # elif size == 224:
  #   cm = [x for x in os.walk('CM224')][0][2]
  #   hs = [x for x in os.walk('HS224')][0][2]
  #   hc = [x for x in os.walk('HC224')][0][2]
  
  # else:
  #   return 'invalid size'
  cm = [x for x in os.walk('CM100')][0][2]
  hs = [x for x in os.walk('HS100')][0][2]
  hc = [x for x in os.walk('HC100')][0][2]
  
  # cm = ['CM100/' + cm[x] for x in range(len(cm))]
  # hs = ['HS100/' + hs[x] for x in range(len(hs))]
  # hc = ['HC100/' + hc[x] for x in range(len(hc))]
  

  cm = ['CM100/' + cm[x] for x in range(len(cm))]
  hs = ['HS100/' + hs[x] for x in range(len(hs))]
  hc = ['HC100/' + hc[x] for x in range(len(hc))]
  
  cm_label = [0]*len(cm)
  hs_label = [1]*len(hs)
  hc_label = [2]*len(hc)

  dataset = cm+hs+hc
  labels = cm_label + hs_label + hc_label

  return dataset, labels

In [ ]:
def read_images(dataset):
  images = []
  
  for filepath in dataset:
    images.append(cv2.imread(filepath))
  
  return images

In [ ]:
dataset, labels = load_dataset()

In [ ]:
print(labels.count(0), labels.count(1), labels.count(2))

500 500 500


In [ ]:
dataset = read_images(dataset)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.3, random_state=42)

In [ ]:
X_train = np.array(X_train) / 255.0
X_test = np.array(X_test) / 255.0
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
len(y_train) - np.count_nonzero(y_train) # 397, 27.6%
# np.count_nonzero(y_train==1) # 365, 30%
# np.count_nonzero(y_train==2) # 334, 32.8%

352

In [ ]:
X_train.shape

(1050, 100, 100, 3)

In [ ]:
X_train[0]

In [ ]:
y_train, y_test = np_utils.to_categorical(y_train), np_utils.to_categorical(y_test)

# Define Model

In [ ]:
# #creating the network architecture
# cnn = models.Sequential([
#     layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(100, 100, 3)),
#     layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'),
#     layers.MaxPooling2D((3, 3)),
    
#     layers.Dropout(0.6),
    
#     layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
#     layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
#     layers.MaxPooling2D((2, 2)),
    
#     layers.Dropout(0.5),
    
#     layers.Flatten(),
#     layers.Dense(512, activation='relu'),
#     layers.Dropout(0.3),
#     layers.Dense(256, activation='relu'),
#     layers.Dense(3, activation='softmax')
# ])

# cnn.compile(optimizer='sgd',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [ ]:
#creating the network architecture
cnn = models.Sequential([
    layers.Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.Conv2D(filters=1024, kernel_size=(3, 3), activation='relu'),
    
    # layers.BatchNormalization(),
    layers.MaxPooling2D((3, 3)),
    
    layers.Dropout(0.6),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.04)),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.03)),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.02)),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

cnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 98, 98, 1024)      28672     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 96, 96, 1024)      9438208   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 1024)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 1024)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1048576)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               536871424 
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)              

# Training

In [ ]:
cnn_history = cnn.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

# Results

In [ ]:
#plotting loss graph
plt.figure()
plt.title('Loss')
plt.plot(cnn_history.history['loss'])
plt.plot(cnn_history.history['val_loss'])
plt.legend(['train', 'test'])
plt.ylim([0, 18])
plt.show()

#plotting accuracy graph
plt.figure()
plt.title('Accuracy')
plt.plot(cnn_history.history['accuracy'])
plt.plot(cnn_history.history['val_accuracy'])
plt.legend(['train', 'test'])
plt.ylim([0, 1])
plt.show()

In [ ]:
image_input = layers.Input(shape=(224, 224, 3))

# model = ResNet50(input_tensor=image_input, include_top=True,weights='imagenet')

res = applications.ResNet50(
    include_top=True,
    weights=None,
    input_tensor=image_input,
    input_shape=(224,224,3),
    pooling=None,
    classifier_activation="softmax",
)

last_layer = res.get_layer('avg_pool').output
x = layers.Flatten(name='flatten1')(last_layer)
d1 = layers.Dropout(0.5, name='dropout1')(x)

y = layers.Dense(512, activation='relu', name='flatten2')(d1)
d2 = layers.Dropout(0.5, name='dropout2')(y)

z = layers.Dense(256, activation='relu', name='flatten3')(d2)
d3 = layers.Dropout(0.5, name='dropout3')(z)

out = layers.Dense(3, activation='softmax', name='output_layer')(d3)
resnet = models.Model(inputs=image_input,outputs= out)

resnet.compile(optimizer = optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
resnet.summary()